In [167]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("model/")
from diffusion import GaussianDiffusion, DiffusionImageAPI
import torch
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
from PIL import Image
import requests

url = "https://www.themoviedb.org/t/p/w1280/6oom5QYQ2yQTMJIbnvbkBL9cHo6.jpg"
image = Image.open(requests.get(url, stream=True).raw)
w, h = image.size
image = image.resize((w // 16, h // 16))
#image = image.resize((16, 16))
w, h = image.size
print(f"[{w}x{h}]")
image
image.save("./test.jpg")


[80x120]


In [174]:
diffusion = GaussianDiffusion(
  model=None,
  noise_steps=256,
  beta_0=1e-4,
  beta_T=0.02,
  image_size=(w, h),
  schedule="linear"
)

In [187]:
imageAPI = DiffusionImageAPI(diffusion)
timesteps = np.linspace(0, 255, 255).astype(int)
print(timesteps)

images = imageAPI.get_noisy_images(image, time_steps=timesteps)
images[0] = image
# save as gif
images[0].save("test.gif", save_all=True, append_images=images[1:], duration=100, loop=0)

#for i in range(len(images)):
#  images[i].save(f"./diffusion_samples/test_{i}.jpg")
#  plt.subplot(2, 5, i + 1)
#  plt.imshow(images[i])
#  plt.axis("off")


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [172]:
len(torch.Size([2, 1, 120, 80, 3]))


5

In [173]:
image = image_copy
image = torch.from_numpy(np.array(image))
image = diffusion.normalize_image(image)
t = diffusion.sample_time_steps(1)
print(t)
image = diffusion.apply_noise(image, t)
imageAPI.tensor_to_image(diffusion.denormalize_image(image.squeeze(0)))


NameError: name 'image_copy' is not defined